In [1]:
import json
import jsonlines


In [2]:
# with open("/root/code/ChatGLM_PEFT/main/data/sample_dev_pred.json", 'r', encoding='utf-8') as f:
#     obj1 = json.load(f)
#     print(type(obj1['answer']))

In [3]:

# /root/code/ChatGLM_PEFT/main/data/hotpot_dev_distractor_v1.json
dev = []
with open("/root/code/ChatGLM_PEFT/main/data/temp_dev_content.json", 'r', encoding='utf-8') as file:
        dev = json.load(file)
init_dev = []
with open("/root/code/ChatGLM_PEFT/main/data/hotpot_dev_distractor_v1.json", 'r', encoding='utf-8') as file:
        init_dev = json.load(file)
print(len(dev))
print(len(init_dev))

7405
7405


In [4]:
# 打开JSONL文件
# with open("/root/code/ChatGLM_PEFT/output.txt", "r", encoding="utf8") as f:

# 读取 .jsonl 文件并将其转换为 Python 对象列表
from json import JSONDecodeError


def read_jsonl(file_path):    
    answer_parse_error_count = 0
    sp_parse_error_count = 0
    data = {
        "answer":{},
        "sp":{}
    }
    
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    line_num = len(lines) 
    print(line_num)
    with open(file_path, 'r', encoding='utf-8') as file:
        item_count = 0
        count_index = 0
        while item_count < (line_num/2)-1 :
            # 读取两行
            answer_line = json.loads(file.readline().strip())
            sp_line = json.loads(file.readline().strip())
            # print(answer_line)
            # print(sp_line)
            # 如果两行都为空，表示文件已经读完，退出循环(错误，出现连续两个空输出就老实了)
            # if not answer_line and not sp_line:
            #     break
            
            answer = ""
            try:
                answer = json.loads(str(answer_line).strip())  # 逐行读取并转换为 Python 对象
                # print(answer)
                # print("answer_type:"+str(type(answer)))
            except TypeError:
                answer = "<|>解析错误<|>"
                # print("答案解析错误")
                answer_parse_error_count += 1
            except JSONDecodeError:
                try:
                    clean_line = answer_line.strip().strip('"').strip().strip('"')
                    answer = clean_line
                except Exception:
                    answer = "<|>解析错误<|>"
                    # print("答案解析错误") 
                    answer_parse_error_count += 1

            # print(answer)    
            data['answer'][dev[count_index]['_id']] = str(answer)    
    
            sp=[]
            try:
                sp = json.loads(sp_line)
                # print(sp)
                # print("sp_type"+str(type(sp)))
                # data.append(obj1) 
            except JSONDecodeError :
                # print(sp_line)
                # 绝大多数格式错误都是少了前面的 [
                sp_line = "[" + str(sp_line)
                try:
                    sp = json.loads(sp_line)
                    # print("JSONDecode_sp_type" + str(type(sp)))
                    # data.append(obj1)   
                except JSONDecodeError:
                    #    其他类型错误，直接舍弃
                    sp_parse_error_count += 1
                    sp=[]
            except TypeError:
                sp_parse_error_count += 1
                sp=[]
            # print(sp)
            data['sp'][dev[count_index]['_id']] = sp
            count_index += 1
            item_count += 1
    # print("answer解析错误率"+str(answer_parse_error_count/item_count))
    # print("sp解析错误率"+str(sp_parse_error_count/item_count))
    print(len(data['sp']))
    return data

# 使用示例
file_path = '/root/code/ChatGLM_PEFT/output.txt'  # 将其替换为你的 .jsonl 文件路径
data = read_jsonl(file_path)

14810
7404


In [5]:
def find_contextObj_by_id(data_list, search_id):
    for item in data_list:
        if item['_id'] == search_id:
            return item['context']
    return None  # 如果未找到

def find_contextSentIndex_by_id(list,search_id,title,letter_index):
    for item in list:
        if item[0] == title:
            for sent_dict in item[1]:
                for key,value in sent_dict.items():
                    if key == letter_index:#通过letter_index去找原句
                        sent =  sent_dict[key]# 通过原句去找index
                        init_context_list = find_contextObj_by_id(init_dev,search_id)
                        for init_context_item in init_context_list:
                            if init_context_item[0] == title:
                                for index in range(len(init_context_item[1])):
                                    if sent.casefold() == init_context_item[1][index].casefold():
                                        return index
                 
    return None

In [6]:
# print(dev)
for key,value in data['sp'].items():
    # print(value)
    new_sp = []
    for item in value:
        if len(item) >=2 :
            # print(item)
            titile = item[0]
            # print(titile)
            letter_index = item[1]
            # print(letter_index)
            context_obj = find_contextObj_by_id(dev,key)  
            # print(type(context_obj[0][1])) 
            if context_obj != None:
                sent = find_contextSentIndex_by_id(context_obj,key,titile,letter_index)
                if sent != None:
                    new_sp_item = [titile,sent]
                    new_sp.append(new_sp_item)
    data['sp'][key] = new_sp
with open("/root/code/ChatGLM_PEFT/main/data/te.json", 'w') as write_f:
	json.dump(data, write_f, indent=4, ensure_ascii=False)
